In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [5]:
EMBEDDING_SIZE=64
HIDDEN_SIZE=256
BATCH_SIZE=32
NUM_STEPS=16
NUM_EPOCHS_INIT_LR=3
NUM_EPOCHS_TOTAL=8
INITIAL_LR=5e0
LR_DECAY_RATE=0.75
MAX_NORM=0.5

In [14]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    initializer_scale=0.2
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
training_nodes = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [15]:
training_outputs = {**summary_nodes, **training_nodes}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 64 == 63):
                print('step: {0}    loss: {1}    gradient norm: {2}     correct words: {3}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['gradient_global_norm'],
                    outputs['num_correct_predictions'],
                ))
                
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('validation perplexity:', np.exp(total_loss / total_batches))
        total_loss, total_batches = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_batches += 1
        print('test perplexity:', np.exp(total_loss / total_batches))

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 64    loss: 8.187310218811035    gradient norm: 2.395899772644043     correct words: 22
step: 128    loss: 7.125223636627197    gradient norm: 0.4549958407878876     correct words: 13
step: 192    loss: 6.89569616317749    gradient norm: 0.691180408000946     correct words: 27
step: 256    loss: 7.569201946258545    gradient norm: 2.4837846755981445     correct words: 29
step: 320    loss: 6.528324127197266    gradient norm: 0.31351301074028015     correct words: 32
step: 384    loss: 7.0342512130737305    gradient norm: 0.7642437219619751     correct words: 33
step: 448    loss: 6.986194610595703    gradient norm: 1.3960965871810913     correct words: 37
step: 512    loss: 7.007909774780273    gradient norm: 0.4218199849128723     correct words: 27
step: 576    loss: 7.155097007751465    gradient norm: 1.537655234336853     correct words: 46
step: 640    loss: 6.88791561126709    gradient norm: 1.2366260290145874     correct words: 30
step: 704    loss: 6.382382869720459    grad

validation perplexity: 286.566447698
test perplexity: 281.168458673
step: 64    loss: 5.3692474365234375    gradient norm: 0.49441781640052795     correct words: 90
step: 128    loss: 5.485985279083252    gradient norm: 0.5218965411186218     correct words: 78
step: 192    loss: 5.417150974273682    gradient norm: 0.4896920919418335     correct words: 80
step: 256    loss: 5.17519998550415    gradient norm: 0.44671931862831116     correct words: 105
step: 320    loss: 5.040509223937988    gradient norm: 0.6598420739173889     correct words: 100
step: 384    loss: 5.237536430358887    gradient norm: 0.4720807373523712     correct words: 95
step: 448    loss: 5.519776821136475    gradient norm: 0.8687852025032043     correct words: 80
step: 512    loss: 5.26367712020874    gradient norm: 0.5292115807533264     correct words: 93
step: 576    loss: 5.235969066619873    gradient norm: 0.5788769721984863     correct words: 96
step: 640    loss: 4.965902805328369    gradient norm: 0.669863820

step: 1792    loss: 4.744772434234619    gradient norm: 0.6065486073493958     correct words: 103
validation perplexity: 151.870021887
test perplexity: 145.589816687
step: 64    loss: 4.813520431518555    gradient norm: 0.5433322787284851     correct words: 106
step: 128    loss: 4.9655280113220215    gradient norm: 0.6324458122253418     correct words: 108
step: 192    loss: 4.741883754730225    gradient norm: 0.5546649098396301     correct words: 108
step: 256    loss: 4.672945022583008    gradient norm: 0.5560429692268372     correct words: 109
step: 320    loss: 4.596774578094482    gradient norm: 0.5572349429130554     correct words: 126
step: 384    loss: 4.660979270935059    gradient norm: 0.5391877889633179     correct words: 104
step: 448    loss: 4.956696510314941    gradient norm: 0.6182261109352112     correct words: 100
step: 512    loss: 4.788901329040527    gradient norm: 0.5687500834465027     correct words: 110
step: 576    loss: 4.6385626792907715    gradient norm: 0.